In [1]:
import csv
import time
import json
import requests
import pandas as pd
from bs4 import BeautifulSoup

In [2]:
# https://github.com/Ledka17/Parsing_banki_ru/blob/master/Parsing_banki_ru.ipynb
# https://github.com/kotikkonstantin/bankiru/blob/master/bankiru_parsing.ipynb
# https://github.com/KKQUEN/banki.ru_reviews_parser/blob/main/parser_banki.ru_ModeratedReviews.ipynb

In [3]:
# список банков

for year in [2023, 2024]:

  df = pd.DataFrame({'place':[], 'name':[], 'rating':[], 'responses':[], 'answers':[]})

  page = 1
  i = 1
  while True:

    url = f'https://www.banki.ru/services/responses/?date={year}&page={page}'
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    items = soup.find_all("script", {"type": "application/ld+json"})

    if len(items) == 1:
      break

    for item in items[:50]:
      data = json.loads(item.text)
      if 'name' in data:
        place = i
        name = data['name']
        rating = data['aggregateRating']['ratingValue']
        responses = data['aggregateRating']['ratingCount']
        answers = data['aggregateRating']['reviewCount']

        df.loc[len(df)] = [i, name, rating, responses, answers]
        i += 1
    page += 1

  df.to_csv(f'banki_banks_{year}.csv', index=False)

In [4]:
# отзывы

base_url = 'https://www.banki.ru'
page = 1

with open('banki_responses.csv', "w", encoding='utf-8') as w_file:
  file_writer = csv.writer(w_file, delimiter = ",", lineterminator="\n")
  file_writer.writerow(['url', 'date_response', 'time_response', 'bank_name', 'username', 'user_city', 'review_title', 'review_text'])

  while True:

    url = f'https://www.banki.ru/services/responses/list/?page={page}&type=all'
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    items = soup.find_all('a', attrs={'class': 'link-simple',
                                      'data-gtm-click': '{"event":"GTM_event","eventCategory":"ux_data","eventAction":"click_responses_response_user_rating_banks"}'})
    for item in items:

      url_resp = base_url+item['href']

      response_resp = requests.get(url_resp)
      soup_resp = BeautifulSoup(response_resp.content, 'html.parser')

      date_time = soup_resp.find('span', {'class':'l10fac986'}).text
      date = date_time[4:14]
      time = date_time[15:]
      bank_name = soup_resp.find('img', {'class':'lazy-load'})['alt']
      review_title = soup_resp.find('h1', {'class':'text-header-0 le856f50c'}).text
      review_text = soup_resp.find("div", {"class":"lf4cbd87d ld6d46e58 lfd76152f"}).text
      username = soup_resp.find('span', {'class':'l17191939'}).text
      user_city = soup_resp.find('span', {'class':'l3a372298'}).text


      file_writer.writerow([url_resp, date, time, bank_name, username, user_city, review_title, review_text])


    w_file.flush()
    page += 1

    if page == 3:   # тут потом переписать - ограничение по дате!
      break